![(book cover)](https://covers.oreillystatic.com/images/0636920167433/cat.gif "(book cover)")
# "Programming Quantum Computers" by O'Reilly Media -  [book info](http://shop.oreilly.com/product/0636920167433.do)  - [all code samples](https://oreilly-qc.github.io)

## Code samples for Chapter 5
These code samples were written by Mariia Mykhailova.

### Example 5-1: Integer increment-by-one operation

#### Example 5-1, part 1: Integer increment-by-one operation (circuit implementation)

In [1]:
// Example 5-1: Integer increment-by-one operation (circuit implementation)

open Microsoft.Quantum.Diagnostics;

// Circuit implementation of modular increment operation
operation CircuitIncrement (register : Qubit[]) : Unit is Adj {
    for i in Length(register) - 1 .. -1 .. 0 {
        Controlled X(register[0 .. i - 1], register[i]);
    }
}

operation CircuitIncrementAndDecrement () : Unit {
    // allocate the qubit register
    use register = Qubit[4];
    // initialize the inputs
    X(register[0]);
    H(register[2]);
    T(register[2]);

    DumpMachine(());
    // note that at this point non-zero amplitudes are at the basis states |1❭ and |5❭

    Message("Increment (circuit implementation)");
    CircuitIncrement(register);
    DumpMachine(());
    // note that after incrementing the same amplitudes shifted to the basis states |5❭ and |6❭

    // decrement is an adjoint of increment, so we don't need to implement it separately
    Message("Decrement (curcuit implementation)");
    Adjoint CircuitIncrement(register);
    DumpMachine(());

    // make sure the qubits are back to the |0❭ state
    ResetAll(register);
}

CircuitIncrement, CircuitIncrementAndDecrement

Note the use of `%config` below: setting `dump.truncateSmallAmplitudes` to `true` hides the basis states of a state vector with near-zero measurement probabilities.

In [2]:
%config dump.truncateSmallAmplitudes=true
%simulate CircuitIncrementAndDecrement

|1⟩	0.7071067811865476 + 0𝑖
|5⟩	0.5000000000000001 + 0.5000000000000001𝑖

Increment (circuit implementation)


|2⟩	0.7071067811865476 + 0𝑖
|6⟩	0.5000000000000001 + 0.5000000000000001𝑖

Decrement (curcuit implementation)


|1⟩	0.7071067811865476 + 0𝑖
|5⟩	0.5000000000000001 + 0.5000000000000001𝑖

()

#### Example 5-1, part 2: Integer increment-by-one operation (using library operation)

In [3]:
// Example 5-1: Integer increment-by-one operation (using library operation)

open Microsoft.Quantum.Arithmetic;
open Microsoft.Quantum.Diagnostics;

operation LibraryIncrementAndDecrement () : Unit {
    // allocate the qubit register
    use register = Qubit[4];
    // initialize the inputs
    X(register[0]);
    H(register[2]);
    T(register[2]);

    DumpMachine(());
    // note that at this point non-zero amplitudes are at the basis states |1❭ and |5❭

    // Increment the register using library implementation of the increment from Microsoft.Quantum.Arithmetic
    Message("Increment (library implementation)");
    IncrementByInteger(1, LittleEndian(register));
    DumpMachine();

    Message("Decrement (library implementation)");
    IncrementByInteger(-1, LittleEndian(register));
    DumpMachine();

    // make sure the qubits are back to the |0❭ state
    ResetAll(register);
}

LibraryIncrementAndDecrement

In [4]:
%simulate LibraryIncrementAndDecrement

|1⟩	0.7071067811865476 + 0𝑖
|5⟩	0.5000000000000001 + 0.5000000000000001𝑖

Increment (library implementation)


|2⟩	0.7071067811865482 + -2.465190328815662E-32𝑖
|6⟩	0.5000000000000007 + 0.5000000000000006𝑖

Decrement (library implementation)


|1⟩	0.707106781186549 + 5.551115123125777E-17𝑖
|5⟩	0.5000000000000014 + 0.5000000000000011𝑖

()

### Example 5-2: Adding two quantum integers

In [5]:
%package Microsoft.Quantum.Numerics

Adding package Microsoft.Quantum.Numerics: done!

Microsoft.Quantum.Standard::0.15.2101125897, Microsoft.Quantum.Standard.Visualization::0.15.2101125897, Microsoft.Quantum.Numerics::0.15.2101125897

In [6]:
// Example 5-2: Adding two quantum integers

open Microsoft.Quantum.Diagnostics;

// Open namespace which defines arithmetic operations
open Microsoft.Quantum.Arithmetic;

operation QuantumAddition () : Unit {
    use (a, b) = (Qubit[4], Qubit[2]);
    // Initialize the inputs
    X(a[0]);
    H(a[2]);
    T(a[2]);
    Message("Register a");
    DumpRegister((), a);
    // At this point non-zero amplitudes of register a are at the basis states |1❭ and |5❭

    X(b[0]);
    H(b[1]);
    T(b[1]);
    Message("Register b");
    DumpRegister((), b);
    // At this point non-zero amplitudes of register b are at the basis states |1❭ and |3❭

    // The library operation AddI will add the first parameter to the second one
    AddI(LittleEndian(b), LittleEndian(a));

    // Note that now the registers a and b are entangled, so you can not look at just the state of the register b.
    Message("State of the system after computing a += b");
    DumpMachine();
    // The output of this operation is a bit tricky to decipher, since it represents both registers a and b.
    // Register a is the lowest 4 bits of the overall state, and register b is the highest 2 bits.
    // Thus, the non-zero amplitude 0.5 of the basis state |18❭ corresponds to the little-endian bit mask 010010,
    // which can be split up into registers a and b as 0100 10, 
    // which in turn is converted to decimal numbers 2 and 1:
    // the first number is the basis state of register a, storing the sum of basis states |1❭ and |1❭,
    // and the second number is the input state of register b (|1❭).
    // The amplitude of this state is a product of the amplitudes of the basis states |1❭ and |1❭ that were added, 1/sqrt(2) * 1/sqrt(2).
    // The rest of the states can be traced in a similar way.

    // Make sure the qubits are back to the |0❭ state
    ResetAll(a + b);
}

QuantumAddition

In [7]:
%simulate QuantumAddition

Register a


|1⟩	0.7071067811865475 + 0𝑖
|5⟩	0.5 + 0.5𝑖

Register b


|1⟩	0.4999999999999999 + 0.4999999999999999𝑖
|3⟩	1.7431527984210487E-32 + 0.7071067811865476𝑖

State of the system after computing a += b


|18⟩	0.5000000000000001 + 0𝑖
|22⟩	0.35355339059327384 + 0.35355339059327384𝑖
|52⟩	0.35355339059327384 + 0.35355339059327384𝑖
|56⟩	1.232595164407831E-32 + 0.5000000000000002𝑖

()

### Example 5-3: Some interesting arithmetic

In [8]:
// Example 5-3: Some interesting arithmetic

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arithmetic;

operation AddBSquared () : Unit {
    use (a, b) = (Qubit[4], Qubit[2]);
    // Initialize the inputs
    X(a[0]);
    H(a[2]);
    T(a[2]);
    Message("Register a");
    DumpRegister((), a);
    // At this point non-zero amplitudes of register a are at the basis states |1❭ and |5❭

    X(b[0]);
    H(b[1]);
    T(b[1]);
    Message("Register b");
    DumpRegister((), b);
    // At this point non-zero amplitudes of register b are at the basis states |1❭ and |3❭

    // Allocate extra qubits to hold the value of b² temporarily;
    // note that the register has to have twice the number of qubits in register b.
    use anc = Qubit[4];
    within {
        // Compute anc = b * b
        SquareI(LittleEndian(b), LittleEndian(anc));
    } apply {
        // Compute a += anc
        AddI(LittleEndian(anc), LittleEndian(a));
    }
    
    // Note that now the registers a and b are entangled, so you can not look at just the state of the register b.
    Message("State of the system after computing a += b * b");
    DumpMachine();
    // The output of this operation can be deciphered in a same way as the previous one:
    // basis states |18❭ and |22❭ remain unchanged, since they correspond to the number 1 written in register b;
    // basis state |58❭ corresponds to little-endian binary 010111, which represents b = 3 and a + b * b = 1 + 3² = 10,
    // and basis state |62❭ is binary 011111, which is b = 3 and a + b * b = 5 + 3² = 14. 

    // Make sure the qubits are back to the |0❭ state
    ResetAll(a + b);
}

AddBSquared

In [9]:
%simulate AddBSquared

Register a


|1⟩	0.7071067811865475 + 0𝑖
|5⟩	0.5 + 0.5𝑖

Register b


|1⟩	0.4999999999999999 + 0.4999999999999999𝑖
|3⟩	1.7431527984210487E-32 + 0.7071067811865476𝑖

State of the system after computing a += b * b


|18⟩	0.5000000000000001 + 0𝑖
|22⟩	0.35355339059327384 + 0.35355339059327384𝑖
|58⟩	0.35355339059327384 + 0.35355339059327384𝑖
|62⟩	1.232595164407831E-32 + 0.5000000000000002𝑖

()

### Example 5-4: Quantum-conditional execution

In [10]:
// Example 5-4: Quantum-conditional execution

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arithmetic;

operation QuantumConditionalExecution () : Unit {
    use (a, b) = (Qubit[3], Qubit[3]);
    // Initialize the inputs
    X(a[0]);
    H(a[2]);
    Message("Register a");
    DumpRegister((), a);
    // At this point non-zero amplitudes of register a are at the basis states |1❭ and |5❭

    X(b[0]);
    H(b[1]);
    T(b[1]);
    Message("Register b");
    DumpRegister((), b);
    // At this point non-zero amplitudes of register b are at the basis states |1❭ and |3❭

    // a -= 3
    IncrementByInteger(-3, LittleEndian(a));

    // if (a < 0) then b++;
    (Controlled IncrementByInteger)([a[2]], (1, LittleEndian(b)));

    // a += 3
    IncrementByInteger(3, LittleEndian(a));

    // Note that now the registers a and b are entangled, so you can not look at just the state of the register b.
    Message("State of the system after the computation");
    DumpMachine();

    // Make sure the qubits are back to the |0❭ state
    ResetAll(a + b);
}

QuantumConditionalExecution

In [11]:
%simulate QuantumConditionalExecution

Register a


|1⟩	0.7071067811865476 + 0𝑖
|5⟩	0.7071067811865476 + 0𝑖

Register b


|1⟩	0.7071067811865475 + 0𝑖
|3⟩	0.5 + 0.5𝑖

State of the system after the computation


|13⟩	0.5000000000000014 + -2.699809788466976E-17𝑖
|17⟩	0.5000000000000014 + -3.835274586632034E-17𝑖
|29⟩	0.35355339059327473 + 0.3535533905932747𝑖
|33⟩	0.35355339059327473 + 0.3535533905932747𝑖

()

### Example 5-5: Phase-encoding the result

In [12]:
// Example 5-5: Phase-encoding the result

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Arithmetic;

operation PhaseEncoding () : Unit {
    use (a, b) = (Qubit[3], Qubit[2]);
    // Initialize the inputs
    H(a[0]);
    H(a[1]);
    Message("Register a");
    DumpRegister((), a);
    // At this point non-zero amplitudes of register a are at the basis states |0❭, |1❭, |2❭, and |3❭

    X(b[0]);
    H(b[1]);
    Message("Register b");
    DumpRegister((), b);
    // At this point non-zero amplitudes of register b are at the basis states |1❭ and |3❭

    Message("Complete state of the system");
    DumpMachine(());

    // a -= 3
    IncrementByInteger(-3, LittleEndian(a));

    // if (a < 0) and b == 1 then flip the phase
    (ControlledOnInt(1, Z))(b, a[2]);

    // a += 3
    IncrementByInteger(3, LittleEndian(a));

    // Note that now the registers a and b are entangled, so you can not look at just the state of the register b.
    Message("State of the system after the computation");
    DumpMachine(());

    // Make sure the qubits are back to the |0❭ state
    ResetAll(a + b);
}

PhaseEncoding

In [13]:
%simulate PhaseEncoding

Register a


|0⟩	0.5 + 0𝑖
|1⟩	0.5 + 0𝑖
|2⟩	0.5 + 0𝑖
|3⟩	0.5 + 0𝑖

Register b


|1⟩	0.7071067811865476 + 0𝑖
|3⟩	0.7071067811865476 + 0𝑖

Complete state of the system


|8⟩	0.35355339059327384 + 0𝑖
|9⟩	0.35355339059327384 + 0𝑖
|10⟩	0.35355339059327384 + 0𝑖
|11⟩	0.35355339059327384 + 0𝑖
|24⟩	0.35355339059327384 + 0𝑖
|25⟩	0.35355339059327384 + 0𝑖
|26⟩	0.35355339059327384 + 0𝑖
|27⟩	0.35355339059327384 + 0𝑖

State of the system after the computation


|8⟩	-0.3535533905932743 + 1.962615573354723E-17𝑖
|9⟩	-0.3535533905932744 + 6.364057111333954E-17𝑖
|10⟩	-0.35355339059327445 + 2.2007207689896168E-17𝑖
|11⟩	0.35355339059327423 + -4.738173134917614E-17𝑖
|24⟩	0.3535533905932742 + -3.3503943541361674E-17𝑖
|25⟩	0.35355339059327434 + -1.9626155733547217E-17𝑖
|26⟩	0.3535533905932743 + 4.7381731349176125E-17𝑖
|27⟩	0.35355339059327434 + 4.738173134917614E-17𝑖

()

### Example 5-6: Logic using CNOT gates

In [14]:
// Example 5-6: Logic using CNOT gates

open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Measurement;

operation CNOTLogic () : Unit {
    Message("c = ~c");
    use (a, b, c) = (Qubit(), Qubit(), Qubit());
    Message($"Input: c = {M(c)}");
    X(c);
    Message($"Output: c = {M(c)}");
    Reset(c);
    
    // ------------------------------------------------------
    Message("");
    Message("if (b) then c = ~c");
    // initialize input
    X(b);
    Message($"Input: b = {M(b)}, c = {M(c)}");

    CNOT(b, c);
    Message($"Output: b = {M(b)}, c = {M(c)}");

    ResetAll([b, c]);
    
    // ------------------------------------------------------
    Message("");
    Message("if (a and b) then c = ~c");
    // initialize input
    X(a);
    X(b);
    Message($"Input: a = {M(a)}, b = {M(b)}, c = {M(c)}");

    (Controlled X)([a, b], c);
    Message($"Output: a = {M(a)}, b = {M(b)}, c = {M(c)}");

    ResetAll([a, b, c]);
}

CNOTLogic

In [15]:
%simulate CNOTLogic

c = ~c
Input: c = Zero
Output: c = One

if (b) then c = ~c
Input: b = One, c = Zero
Output: b = One, c = One

if (a and b) then c = ~c
Input: a = One, b = One, c = Zero
Output: a = One, b = One, c = One


()